In [ ]:
import numpy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import utils
# load ascii text and covert to lowercase
filename = "AllD/Stalker 3v1.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

# Сортируем по буквам
chars=list(['\n','\xa0','-',' ',',','.','а','б','в','г','д','е','ж','з','и','й','к','л','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ъ','ы','ь','э','ю','я','ё'])
#chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

In [ ]:
import os


os.environ['CUDA_VISIBLE_DEVICES'] = '1' #-1 не использовать gpu

In [ ]:
!nvidia-smi

In [ ]:
# режем на кусочки последовательностей 100 и больше мб
seq_length = 200
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Всего шаблончиков: ", n_patterns)

In [ ]:
# Решейпим в тензор туды - сюды
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = utils.to_categorical(dataY)

In [ ]:

# Задаем модель Долгой краткосрочной памяти
model = models.Sequential()
model.add(layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Сохраняем веса енто нужно только при длительном обучении
filepath="Stalker3v1LOWMODEL-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:

model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

### Генерируем текст

In [ ]:
# Загружаем сохраненное
filename = "Stalker3v1LowS b-124lost-3015.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
import sys
# выбираем рандом плод и смотрим как оно
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Кусок:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")